# Install pyG


In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

# Data processing

In [2]:
data_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/0.txt'
ans_path = '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/0_score.txt'

In [3]:
ans = {}
with open(ans_path) as f:
  line = f.readline()
  while line:
    key, value = line.split()
    key = eval(key)
    ans[key] = eval(value)
    line = f.readline()

In [4]:
import torch
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import from_networkx

G = nx.read_edgelist(data_path, nodetype = int)
edge_index = from_networkx(G).edge_index
x = torch.tensor([[[G.degree(node),1,1] for node in G.nodes]], dtype=torch.float)
y = torch.tensor([ans[node] for node in G.nodes], dtype=torch.float)
data = Data(edge_index=edge_index, x=x, y=y)
# train_loader = DataLoader(data, batch_size=64, shuffle=True) # TODO: split test data

# train

In [5]:
import torch
from torch.nn import Linear, GRU, ReLU
from torch_geometric.nn import GCNConv, max_pool
from torch.nn import Sequential as Seq

# TODO: torch_geometric.nn.models.GAE
class Net(torch.nn.Module):
  def __init__(self, L):
    super(Net, self).__init__()
    self.L = L-1 # encoder layer number
    self.data_in = Seq(Linear(3, 4), ReLU()) # TODO: number of feature
    self.n_aggregation = GCNConv(4, 4)
    self.combine = GRU(4, 4, bias=False)
    self.data_out = Seq(Linear(4, 5), ReLU(), Linear(5,1), ReLU())

  def forward(self, x, edge_index):
    # encoder
    h = self.data_in(x)
    h = h / torch.norm(h) # dimension
    layer = []
    layer.append(h)
    for _ in range(self.L):
      hn = self.n_aggregation(h, edge_index)
      h, hn = self.combine(h, hn)
      h = h / torch.norm(h)
      layer.append(h)
    # z = max_pool(layer, data)
    z = h

    # decoder
    y = self.data_out(z)
    y = torch.squeeze(y)
    # print(y)
    return y

In [ ]:
model = Net(5)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def g(x):
  return 1 / (1+1/torch.exp(x))

for epoch in range(110):
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  print(out)
  yij = torch.stack([g(out[i] - out[j]) for i, j in G.edges])
  bij = torch.tensor([g(data.y[i] - data.y[j]) for i, j in G.edges])
  loss = criterion(yij, bij) * 10  # Compute the loss.
  loss.backward()  # Derive gradients.
  print(f'Epoch {epoch}:{loss}')
  optimizer.step()  # Update parameters based on gradients.
  optimizer.zero_grad()  # Clear gradients.


In [7]:
bij

tensor([0.5173, 0.5188, 0.5181,  ..., 0.5000, 0.5000, 0.5000])

In [8]:
yij

tensor([0.5004, 0.5005, 0.5004,  ..., 0.5000, 0.5000, 0.5000],
       grad_fn=<StackBackward>)

In [12]:
ans

{0: 0.09417453090592563,
 1: 0.05397079661985897,
 2: 0.04434365787783783,
 3: 0.022325672571532364,
 4: 0.0764376504965615,
 5: 0.092789552991686,
 6: 0.05002370607942536,
 7: 0.007795097020580318,
 8: 0.017047485161806413,
 9: 0.04654708044354408,
 10: 0.02493647915051651,
 11: 0.032833485490133256,
 12: 0.01903432460894745,
 13: 0.006959679409660805,
 14: 0.016435682466714036,
 15: 0.008363920362448115,
 16: 0.0126680081689472,
 17: 0.03270924166834739,
 18: 0.003599594295941664,
 19: 0.013135099913657314,
 20: 0.021906374682077918,
 21: 0.009681283611761623,
 22: 0.022440748219730196,
 23: 0.016164470334169543,
 24: 0.02241169720285693,
 25: 0.010071138351199114,
 26: 0.017189263996478257,
 27: 0.010285104459068446,
 28: 0.01186975194326765,
 29: 0.006926903459439328,
 30: 0.006891359206131455,
 31: 0.0029940131840764253,
 32: 0.019544721161274477,
 33: 0.019743023423627673,
 34: 0.020597983896156084,
 35: 0.015982200974103133,
 36: 0.005392820151614545,
 37: 0.006468477368543176,
